https://medium.com/@daniel820710/%E5%88%A9%E7%94%A8keras%E5%BB%BA%E6%A7%8Blstm%E6%A8%A1%E5%9E%8B-%E4%BB%A5stock-prediction-%E7%82%BA%E4%BE%8B-1-67456e0a0b

In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
%matplotlib inline

/home/ailab/anaconda3/envs/keras/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def readTrain():
    train = pd.read_csv("SPY.csv")
    return train

train = readTrain()
train = train.drop(["Date"], axis=1)
train = train.drop(["Close"], axis=1)
train

# TODO: add "is_music" column

,Open,High,Low,Adj Close,Volume
0,43.968700,43.968700,43.750000,27.112253,1003200
1,43.968700,44.250000,43.968700,27.305082,480500
2,44.218700,44.375000,44.125000,27.362904,201300
3,44.406200,44.843700,44.375000,27.652185,529400
4,44.968700,45.093700,44.468700,27.767893,531500
5,44.968700,45.062500,44.718700,27.748577,492100
6,44.968700,45.125000,44.906200,27.748577,596100
7,44.812500,44.812500,44.562500,27.555738,122100
8,44.656200,44.750000,44.531200,27.594305,379600
9,44.781200,45.125000,44.781200,27.729336,19500


In [3]:
def normalize(train):
    train = train.drop(["Date"], axis=1)
    train_norm = train.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
    return train_norm

In [28]:
def buildTrain(train, pastDay=30, futureDay=5):
    X_train, Y_train = [], []
    for i in range(train.shape[0]-futureDay-pastDay):
        X_train.append(np.array(train.iloc[i:i+pastDay]))
        Y_train.append(np.array(train.iloc[i+pastDay:i+pastDay+futureDay]["Adj Close"]))
    return np.array(X_train), np.zeros(np.array(Y_train).shape)
    # TODO: return is_music, instead of return futureDay

In [5]:
def shuffle(X,Y):
    np.random.seed(10)
    randomList = np.arange(X.shape[0])
    np.random.shuffle(randomList)
    return X, Y

In [6]:
def splitData(X,Y,rate):
    X_train = X[int(X.shape[0]*rate):]
    Y_train = Y[int(Y.shape[0]*rate):]
    X_val = X[:int(X.shape[0]*rate)]
    Y_val = Y[:int(Y.shape[0]*rate)]
    return X_train, Y_train, X_val, Y_val

In [25]:

# read SPY.csv
train = readTrain()

# Normalization
train_norm = normalize(train)

# build Data, use last 30 days to predict next 5 days
X_train, Y_train = buildTrain(train_norm, 30, 5)

# shuffle the data, and random seed is 10
X_train, Y_train = shuffle(X_train, Y_train)

# split training data and validation data
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)
print(X_train.shape)
# X_trian: (5710, 30, 10)
# Y_train: (5710, 5, 1)
# X_val: (634, 30, 10)
# Y_val: (634, 5, 1)


AttributeError: 'list' object has no attribute 'shape'

# One to one model  
input shape:  
(5710, 1, 10)  
output shape:  
(5710, 1, 1)  

In [8]:
def buildOneToOneModel(shape):
    #print(shape)
    model = Sequential()
    model.add(LSTM(5, input_length=shape[1], input_dim=shape[2],return_sequences=True))
    # output shape: (1, 1)
    model.add(TimeDistributed(Dense(1)))    # or use model.add(Dense(1))
    model.compile(loss="mse", optimizer="adam")
    model.summary()
    return model

In [9]:
train = readTrain()
train_norm = normalize(train)
# change the last day and next day 
X_train, Y_train = buildTrain(train_norm, 1, 1)
X_train, Y_train = shuffle(X_train, Y_train)
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

# from 2 dimmension to 3 dimension
Y_train = Y_train[:,np.newaxis]
Y_val = Y_val[:,np.newaxis]
print(X_train.shape)
print(Y_train.shape)
model = buildOneToOneModel(X_train.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=10, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])


(5710, 1, 6)
(5710, 1, 1)


/home/ailab/anaconda3/envs/keras/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  after removing the cwd from sys.path.
/home/ailab/anaconda3/envs/keras/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(5, return_sequences=True, input_shape=(1, 6))`
  after removing the cwd from sys.path.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1, 5)              240       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 1, 1)              6         
Total params: 246
Trainable params: 246
Non-trainable params: 0
_________________________________________________________________
Train on 5710 samples, validate on 634 samples
Epoch 1/1000
5710/5710 [==============================] - 1s 131us/step - loss: 0.0397 - val_loss: 0.0796
Epoch 2/1000
5710/5710 [==============================] - 0s 16us/step - loss: 0.0272 - val_loss: 0.0507
Epoch 3/1000
5710/5710 [==============================] - 0s 16us/step - loss: 0.0166 - val_loss: 0.0321
Epoch 4/1000
5710/5710 [==============================] - 0s 16us/step - loss: 0.0082 - val_loss: 0.0171
Epoch 5/1000
5710/5710 [==============================] - 0s 17us/step -

Epoch 69/1000
5710/5710 [==============================] - 0s 14us/step - loss: 8.7350e-05 - val_loss: 5.6908e-04
Epoch 70/1000
5710/5710 [==============================] - 0s 15us/step - loss: 8.5389e-05 - val_loss: 5.5755e-04
Epoch 71/1000
5710/5710 [==============================] - 0s 16us/step - loss: 8.3709e-05 - val_loss: 4.7819e-04
Epoch 72/1000
5710/5710 [==============================] - 0s 15us/step - loss: 8.2500e-05 - val_loss: 5.0618e-04
Epoch 73/1000
5710/5710 [==============================] - 0s 14us/step - loss: 8.0312e-05 - val_loss: 4.7849e-04
Epoch 74/1000
5710/5710 [==============================] - 0s 15us/step - loss: 7.8201e-05 - val_loss: 4.9652e-04
Epoch 75/1000
5710/5710 [==============================] - 0s 14us/step - loss: 7.6950e-05 - val_loss: 4.7023e-04
Epoch 76/1000
5710/5710 [==============================] - 0s 15us/step - loss: 7.5158e-05 - val_loss: 4.9335e-04
Epoch 77/1000
5710/5710 [==============================] - 0s 15us/step - loss: 7.3575e-

KeyboardInterrupt: 

# Many to one model  
input shape:  
(5710, 1, 10)  
output shape:  
(5710, 1, 1)  

In [29]:
def buildManyToOneModel(shape):
    model = Sequential()
    model.add(LSTM(5, input_length=shape[1], input_dim=shape[2]))
    # output shape: (1, 1)
    model.add(Dense(1))
    model.compile(loss="mse", optimizer="adam", metrics=['binary_accuracy'])
    model.summary()
    return model

In [30]:
train = readTrain()
train_norm = normalize(train)
# change the last day and next day 
X_train, Y_train = buildTrain(train_norm, 30, 1)
X_train, Y_train = shuffle(X_train, Y_train)
# because no return sequence, Y_train and Y_val shape must be 2 dimension
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)


print(X_train.shape)
print(Y_train.shape)

model = buildManyToOneModel(X_train.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=10, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])

(5684, 30, 6)
(5684, 1)


/home/ailab/anaconda3/envs/keras/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ailab/anaconda3/envs/keras/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(5, input_shape=(30, 6))`
  This is separate from the ipykernel package so we can avoid doing imports until


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 5)                 240       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 6         
Total params: 246
Trainable params: 246
Non-trainable params: 0
_________________________________________________________________
Train on 5684 samples, validate on 631 samples
Epoch 1/10
5684/5684 [==============================] - 2s 296us/step - loss: 0.0017 - binary_accuracy: 1.0000 - val_loss: 1.5745e-04 - val_binary_accuracy: 1.0000
Epoch 2/10
5684/5684 [==============================] - 1s 96us/step - loss: 3.1508e-04 - binary_accuracy: 1.0000 - val_loss: 2.5965e-04 - val_binary_accuracy: 1.0000
Epoch 3/10
5684/5684 [==============================] - 1s 99us/step - loss: 7.7430e-05 - binary_accuracy: 1.0000 - val_loss: 9.5375e-05 - val_binary_accuracy: 1.

In [31]:
result = model.evaluate(X_val, Y_val)

631/631 [==============================] - 0s 82us/step


In [21]:
result

[0.0026771757174607973, 0.0]